In [1]:
import grpc
from concurrent import futures
import time
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize success flag
all_steps_successful = True

# Import the necessary modules for gRPC
def test_grpc_imports():
    global all_steps_successful
    try:
        import grpc
        from concurrent import futures
        logger.info("Step 1: Importing grpc modules passed.")
    except Exception as e:
        all_steps_successful = False
        logger.error(f"Step 1 failed: {str(e)}")

# Define a simple gRPC service in Python
class SimpleServiceServicer:
    def SimpleMethod(self, request, context):
        # Expecting request.name as a string, returning a SimpleResponse
        return SimpleResponse(message="Hello, " + request.name)

# Dummy request and response classes for testing
class SimpleRequest:
    def __init__(self, name):
        self.name = name

class SimpleResponse:
    def __init__(self, message):
        self.message = message

def simple_method_handler(behavior, request_deserializer, response_serializer):
    def handler(request, context):
        request = request_deserializer(request)
        response = behavior(request, context)
        return response_serializer(response)
    return grpc.unary_unary_rpc_method_handler(handler)

def test_grpc_server_client():
    global all_steps_successful
    if all_steps_successful:
        try:
            # Setting up the gRPC server
            server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
            simple_service = SimpleServiceServicer()

            # Manually create method handlers
            method_handlers = {
                'SimpleMethod': simple_method_handler(
                    simple_service.SimpleMethod,
                    request_deserializer=lambda x: SimpleRequest(name=x.decode('utf-8')),
                    response_serializer=lambda x: x.message.encode('utf-8')
                )
            }
            generic_handler = grpc.method_handlers_generic_handler('SimpleService', method_handlers)
            server.add_generic_rpc_handlers((generic_handler,))

            # Start the server
            server.add_insecure_port('[::]:50051')
            server.start()
            logger.info("Step 2: gRPC server started.")

            # Add a short delay to ensure server is ready
            time.sleep(1)

            # Client setup and test
            with grpc.insecure_channel('localhost:50051') as channel:
                stub = channel.unary_unary(
                    '/SimpleService/SimpleMethod',
                    request_serializer=lambda x: x.name.encode('utf-8'),
                    response_deserializer=lambda x: SimpleResponse(message=x.decode('utf-8'))
                )
                response = stub(SimpleRequest(name="World"))
                logger.info(f"Step 3: gRPC client-server communication passed with response: {response.message}")

            # Stop the server
            server.stop(0)
            logger.info("Step 4: gRPC server stopped.")
        except Exception as e:
            all_steps_successful = False
            logger.error(f"gRPC test failed: {str(e)}")

def run_tests():
    test_grpc_imports()
    if all_steps_successful:
        test_grpc_server_client()
    if all_steps_successful:
        logger.info("All tests for the 'grpcio' package completed successfully.")
    else:
        logger.error("Some steps failed. Please check the messages above for details.")

if __name__ == "__main__":
    run_tests()


INFO:__main__:Step 1: Importing grpc modules passed.
INFO:__main__:Step 2: gRPC server started.
INFO:__main__:Step 3: gRPC client-server communication passed with response: Hello, World
INFO:__main__:Step 4: gRPC server stopped.
INFO:__main__:All tests for the 'grpcio' package completed successfully.
